In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from data_utils.data import FaceMaskDetectorData
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F
from models.face_mask_detector import FaceMaskDetector
from settings import *

In [8]:
BATCH_SIZE = 128
ETA = 1e-3
EPOCHS = 5

In [3]:
def compute_accuracy(ps, labels):
    equality = (torch.round(ps) == labels).float()
    acc = equality.mean()
    return acc

In [4]:
def train_step(context, x, y):
    model = context["model"]
    optimizer = context["optimizer"]
    
    optimizer.zero_grad()
    
    ps = model(x)
    ps = ps.squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    loss.backward()
    optimizer.step()
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [5]:
def eval_step(context, x, y):
    model = context["model"]
    
    ps = model(x)
    ps = ps.squeeze()
    loss = F.binary_cross_entropy(ps, y) # nn.BCELoss()(ps, y) 와 같음
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [11]:
def train():
    trainset = FaceMaskDetectorData(mode="train", distortion=True, flip=True)
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    
    testset = FaceMaskDetectorData(mode="test")
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    if CUDA is True:
        model = FaceMaskDetector().cuda()
    else:
        model = FaceMaskDetector()
        
    optimizer = optim.Adam(model.parameters())
    
    context = {
        "model": model,
        "optimizer": optimizer
    }
    
    for e in range(EPOCHS):
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0
        
        model.train()
        
        for x, y in train_loader:
            y = y.float()
            if CUDA is True:
                x = x.cuda()
                y = y.cuda()
            
            loss, acc = train_step(context, x, y)
            train_loss += loss
            train_acc += acc
            
        model.eval()
            
        for x, y in test_loader:
            y = y.float()
            if CUDA is True:
                x = x.cuda()
                y = y.cuda()
            
            with torch.no_grad():
                loss, acc = eval_step(context, x, y)
                test_loss += loss
                test_acc += acc
                
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        print(f"Epochs {e+1}/{EPOCHS}")
        print(f"Train loss: {train_loss:.8f}, train acc: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")
        
        torch.save(model.state_dict(), FACE_MASK_DETECTOR_CKPT)

In [12]:
train()

Epochs 1/5
Train loss: 0.06416446, train acc: 0.9767
Test loss: 0.06560599, test acc: 0.9766
Epochs 2/5
Train loss: 0.01222781, train acc: 0.9957
Test loss: 0.01139956, test acc: 0.9961
Epochs 3/5
Train loss: 0.01128252, train acc: 0.9961
Test loss: 0.00411116, test acc: 0.9980
Epochs 4/5
Train loss: 0.00933823, train acc: 0.9970
Test loss: 0.07452293, test acc: 0.9805
Epochs 5/5
Train loss: 0.00611910, train acc: 0.9978
Test loss: 0.00873989, test acc: 0.9990
